<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/06_MOD17A3HGF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract annual NPP from MODIS

## Import gee and authenticate

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ufKPA1qADUkTt9Oy_OOvCAAuTE3fsGYzcsnzl266zn4&tc=_rySMZ2YhjU_XpNTgC6KyKsAYFf9stpBC391Ynlj_Vw&cc=MmY8sMMA8X_Xbt0HZNy4wEG2tVW3mwGDXLoT-Gwv2vI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VOmGuhHv2WtW_nyWM2PmSeVyZ6gnMttJkNRM2bXKXmkqqaGLd24HGk

Successfully saved authorization token.


## Input arguments for data extraction

In [ ]:
# Area of interest
# aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_NS_buff_5km")
# aoi_name = "MKR_NS_buff_5km"

aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_PACE")
aoi_name = "MKR_PACE"

# GEE layer ID
layer_name = "RS_013"

# Date parameters
start_year = 2012
end_year = 2022

# Range doesn't include the stop value
year_list = ee.List(list(range(start_year, end_year+1)))

## MOD17A3HGF.061: Terra Net Primary Production Gap-Filled Yearly Global 500m



In [ ]:
modis = ee.ImageCollection("MODIS/061/MOD17A3HGF")

## NPP processing

In [ ]:
years = ee.List.sequence(start_year, end_year)

def yearly_NPP(focal_year):
  start = ee.Date.fromYMD(focal_year, 1, 1)
  end = ee.Date.fromYMD(focal_year, 12, 31)
  date_range = ee.DateRange(start, end)
  return modis \
        .select("Npp") \
        .filterDate(date_range) \
        .first() \
        .multiply(0.0001) \
        .set({"name": focal_year}) \
        .clip(aoi)


annual_image = years.map(yearly_NPP)
print(annual_image.getInfo())

## Check an element of list
year_mosaic  = ee.Image(annual_image.get(0))
label = ee.String(year_mosaic.get('name')).getInfo()
print(label)
print(year_mosaic.getInfo())


[{'type': 'Image', 'bands': [{'id': 'Npp', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -3.2768, 'max': 3.2767}, 'dimensions': [86400, 36000], 'crs': 'SR-ORG:6974', 'crs_transform': [463.3127165279165, 0, -20015109.354, 0, -463.3127165279167, 10007554.677003]}], 'properties': {'name': 2012}}, {'type': 'Image', 'bands': [{'id': 'Npp', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -3.2768, 'max': 3.2767}, 'dimensions': [86400, 36000], 'crs': 'SR-ORG:6974', 'crs_transform': [463.3127165279165, 0, -20015109.354, 0, -463.3127165279167, 10007554.677003]}], 'properties': {'name': 2013}}, {'type': 'Image', 'bands': [{'id': 'Npp', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -3.2768, 'max': 3.2767}, 'dimensions': [86400, 36000], 'crs': 'SR-ORG:6974', 'crs_transform': [463.3127165279165, 0, -20015109.354, 0, -463.3127165279167, 10007554.677003]}], 'properties': {'name': 2014}}, {'type': 'Image', 'bands': [{'id': 'Npp', 'data_type': {'typ

## Export data - create task

`filenamePrefix` should be in format: place_layer_timeperiod

In [ ]:
for i in  range(ee.List.length(annual_image).getInfo()):

  output_img =  ee.Image(annual_image.get(i))
  output_name = f"{layer_name}_{aoi_name}_{ee.String(output_img.get('name')).getInfo()}"

  task = ee.batch.Export.image.toDrive(image = output_img,
                                     region = aoi.geometry().bounds(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 30,
                                     maxPixels = 10e12
                                     )
  task.start()
  print("STARTED TASK ", i+1)

STARTED TASK  1
STARTED TASK  2
STARTED TASK  3
STARTED TASK  4
STARTED TASK  5
STARTED TASK  6
STARTED TASK  7
STARTED TASK  8
STARTED TASK  9
STARTED TASK  10
STARTED TASK  11


## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
tasks = ee.batch.Task.list()
for task in tasks[0:ee.List.length(year_list).getInfo()]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)

MXBNCLAION2RWVMZ2OMQM7F2 COMPLETED
TT5H5MXPB2BRHDWMGBCDQ5SS COMPLETED
E2PIQHL7U2X4J2BS6RN3AFRP COMPLETED
D6RDMOYWPX55HNVYIE672H3V COMPLETED
QXMN7OUZHHC2E3GC6FQ4YXFN COMPLETED
X54XI2Q4PIYPI2QHIJ5UQ7RN COMPLETED
JM7NCCO4KSMMVCNR7D7RCKMB COMPLETED
UTMMHMBYAVYFFEVYWKJKFCKS COMPLETED
HLG6KN5ML6PZ5JBBHJA2PN73 COMPLETED
JU4YUDHE4XVQLI5GVEHSRGWE COMPLETED
62LDCIF3OKXJ646BA3O3H3EW COMPLETED
